### Задание 1

In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation
from keras.layers import LSTM, Bidirectional, GRU
from keras.datasets import imdb


import numpy as np
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"


import tensorflow as tf

In [2]:
max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 50 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности


In [3]:
print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [61]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 256))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0, recurrent_activation = 'sigmoid', return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0, recurrent_activation = 'sigmoid')))

model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

Построение модели...


In [62]:
# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [63]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))

Процесс обучения...
500/500 [==============================] - 29s 57ms/step - loss: 0.4201 - accuracy: 0.8048 - val_loss: 0.3548 - val_accuracy: 0.8449


### Задание 2

In [2]:
# построчное чтение из примера с текстом 
with open("Lukyanenko_Dozory.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
print ('Length of text: {} characters'.format(len(text)))

Length of text: 5007312 characters


In [3]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

137 unique characters


In [4]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

a
n
n
o
t


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'annotation На Земле живут «простые люди» и «Иные», к которым относятся маги, волшебники, оборотни, ва'
'мпиры, ведьмы, ведьмаки и проч. Иные делятся на две армии — Светлых (объединенных в Ночной дозор) и Т'
'емных (Дневной дозор). И поскольку простодушия начала ХХ века к концу столетия уже не осталось (а зао'
'дно и идеи Бога), Добро со Злом не борется, а находится в динамическом равновесии. То есть соблюдаетс'
'я баланс Света и Тьмы, и любое доброе магическое воздействие должно уравновешиваться злым. Даже вампи'


In [7]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'annotation На Земле живут «простые люди» и «Иные», к которым относятся маги, волшебники, оборотни, в'
Target data: 'nnotation На Земле живут «простые люди» и «Иные», к которым относятся маги, волшебники, оборотни, ва'


In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 33 ('a')
  expected output: 46 ('n')
Step    1
  input: 46 ('n')
  expected output: 46 ('n')
Step    2
  input: 46 ('n')
  expected output: 47 ('o')
Step    3
  input: 47 ('o')
  expected output: 52 ('t')
Step    4
  input: 52 ('t')
  expected output: 33 ('a')


In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
      tf.keras.layers.GRU(512,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [15]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 137) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           35072     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 512)           2362368   
_________________________________________________________________
dense (Dense)                (64, None, 137)           70281     
Total params: 6,406,025
Trainable params: 6,406,025
Non-trainable params: 0
_________________________________________________________________


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 137)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.9207726


In [20]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS=10

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
774/774 [==============================] - 41s 53ms/step - loss: 2.5695
Epoch 2/10
774/774 [==============================] - 40s 52ms/step - loss: 1.7333
Epoch 3/10
774/774 [==============================] - 40s 52ms/step - loss: 1.5323
Epoch 4/10
774/774 [==============================] - 41s 53ms/step - loss: 1.4624
Epoch 5/10
774/774 [==============================] - 40s 52ms/step - loss: 1.4224
Epoch 6/10
774/774 [==============================] - 40s 52ms/step - loss: 1.3942
Epoch 7/10
774/774 [==============================] - 41s 53ms/step - loss: 1.3726
Epoch 8/10
774/774 [==============================] - 41s 53ms/step - loss: 1.3544
Epoch 9/10
774/774 [==============================] - 40s 52ms/step - loss: 1.3387
Epoch 10/10
774/774 [==============================] - 40s 52ms/step - loss: 1.3258


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            35072     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 512)            2362368   
_________________________________________________________________
dense_1 (Dense)              (1, None, 137)            70281     
Total params: 6,406,025
Trainable params: 6,406,025
Non-trainable params: 0
_________________________________________________________________


In [30]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u"Иной"))

Инойнув остановках. Наверное, можно кивнул. Оказалось, чтобы охотно вынырнули из раста. Вызвав лицами веринг. Он вздохнул, а я посчитал, как мы зашичим, доверяли тогда выя. — Я! — такие улыбкой… что сможет у ней. – Никогда. Нет, они отголовными звуки, из лицу уходы или азавтрали тонкие. Тот это Иная, конечно, не знающая ее не доказывала, мы выехала, но закрывать. Повернулся в карман, хорошо, Антон? Привлек в городе. Но они никакой алкой. Она неохе. В христи, когда это ведет форде Россией. Геннадий намеял, клыек обнаружил взрослые магические желания. Как них з-чта накроем знали, как всего… Три как буквально разумеется, зановыми руками. Запах ловятся, незнами, как газеты долго вынырнули в гараже Света. Воровые девушки этого было, на моих слово богомольцами. Судя по тихоням, как заводные хвостами некоторых раз будет выстрития каких-нибудь изнутрий. – Принассов она демонтро, – сказала она на понимала Анна. – Иди, Максим Мачаши! – Смей на «Аки», ну не зартветив, — выкинив глазу. – Не малейш